In [1]:
import fastf1 as f1
import pandas as pd
import numpy as np
import datetime
import os
from matplotlib import pyplot as plt
import datetime

In [2]:
f1.Cache.enable_cache('cache\\')
seasons = [2023,2024]
drop_labels=['Time', 'PitOutTime', 'PitInTime', 'Sector1Time', 'Sector2Time', 'Sector3Time',
                'Sector1SessionTime', 'Sector2SessionTime', 'Sector3SessionTime',
                'SpeedI1', 'SpeedI2', 'SpeedFL', 'SpeedST', 'IsPersonalBest',
                'FreshTyre', 'LapStartTime',
                'LapStartDate', 'Position', 'Deleted', 'DeletedReason',
                'FastF1Generated', 'IsAccurate']

In [3]:
def update_cache(_year:int):
    f1.Cache.offline_mode(enabled=False)
    e1 = pd.DataFrame(data=f1.get_event_schedule(_year))
    e1 = e1[e1['RoundNumber']!=0]
    for es in e1['EventName']:
        e2 = f1.get_session(_year,gp=es,identifier='R')
        if (e2.date.date()<datetime.date.today()):
                e2.load()

In [8]:
def update_user_data(_season:int):
    df_train=pd.DataFrame()
    f1.Cache.offline_mode(enabled=True)
    for season in _season:
        e1 = pd.DataFrame(data=f1.get_event_schedule(season))
        e1 = e1[e1['RoundNumber']!=0]
        for es in e1['EventName']:
            e2 = f1.get_session(season,es,'R')
            if (e2.date.date()<datetime.date.today()):
                e2.load()
                laps = e2.laps.pick_wo_box().pick_track_status(status='1',how='equals').reset_index(inplace=True)
                weather = laps.get_weather_data().reset_index(inplace=True)
                data = pd.concat([laps,weather],axis=1)
                data['Year'] = e2.event.EventDate.year
                data['GrandPrix'] = e2.event.EventName
                data['LapTime'] = data['LapTime'].dt.total_seconds()
                df_test = pd.DataFrame()
                for drv in data['Driver'].unique():
                    temp = data[data['Driver']==drv].iloc[0:int(0.8*len(data['LapNumber'].unique()))]
                    df_train = pd.concat([df_train,temp])
                
                    #if (os.path.exists('test\\'+str(season)+' '+es+'.csv')==False):
                    temp = data[data['Driver']==drv].iloc[int(0.8*len(data['LapNumber'].unique()))+1:]
                    temp.drop(labels=drop_labels,axis=1,inplace=True)
                    df_test = pd.concat([df_test,temp]).reset_index(drop=True,inplace=True)
                #df_test.drop(labels=drop_labels,axis=1,inplace=True)
                df_test.reset_index(drop=True,inplace=True)
                df_test.to_csv('test\\'+str(season)+' '+es+'.csv',index=False)
    
    df_train.drop(labels=drop_labels,axis=1,inplace=True)
    df_train.reset_index(drop=True,inplace=False)
    df_train.to_csv(f'train\\{_season}.csv',index=None)

In [18]:
update_cache(2020)

core           INFO 	Loading data for Austrian Grand Prix - Race [v3.3.8]
req            INFO 	No cached data found for session_info. Loading data...
_api           INFO 	Fetching session info data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for driver_info. Loading data...
_api           INFO 	Fetching driver list...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for session_status_data. Loading data...
_api           INFO 	Fetching session status data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for lap_count. Loading data...
_api           INFO 	Fetching lap count data...
req            INFO 	Data has been written to cache!
req            INFO 	No cached data found for track_status_data. Loading data...
_api           INFO 	Fetching track status data...
req            INFO 	Data has been written to cache!
req            INFO 	No c

In [9]:
update_user_data([2023,2024])

core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.3.8]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '14', '55', '44', '18', '63', '77', '10', '23', '22', '2', '20', '21', '27', '24', '4', '31', '16', '81']
core           INFO 	Loading data for Saudi Arabian Grand Pri

In [15]:
f1.Cache.offline_mode(enabled=False)
s = f1.get_session(2024,'Bahrain','R',backend='fastf1')
s.load()

core           INFO 	Loading data for Bahrain Grand Prix - Race [v3.3.8]
req            INFO 	Using cached data for session_info
req            INFO 	Using cached data for driver_info
req            INFO 	Using cached data for session_status_data
req            INFO 	Using cached data for lap_count
req            INFO 	Using cached data for track_status_data
req            INFO 	Using cached data for _extended_timing_data
req            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
req            INFO 	Using cached data for car_data
req            INFO 	Using cached data for position_data
req            INFO 	Using cached data for weather_data
req            INFO 	Using cached data for race_control_messages
core           INFO 	Finished loading data for 20 drivers: ['1', '11', '55', '16', '63', '4', '44', '81', '14', '18', '24', '20', '3', '22', '23', '27', '31', '10', '77', '2']


In [147]:
laps = s.laps.pick_wo_box().pick_track_status(status='1',how='equals').pick_driver('VER').reset_index()
weather = laps.get_weather_data().drop(labels=['WindDirection', 'WindSpeed'],axis=1).reset_index()
x = pd.concat([laps,weather],axis=1)
cols = ['AirTemp', 'Humidity', 'Pressure', 'TrackTemp']
x[cols] = x[cols].astype('Float64')
x['AirTemp'].value_counts()
x

,index,Time,Driver,DriverNumber,LapTime,LapNumber,Stint,PitOutTime,PitInTime,Sector1Time,...,DeletedReason,FastF1Generated,IsAccurate,index,Time,AirTemp,Humidity,Pressure,Rainfall,TrackTemp
0,1,0 days 01:03:13.806000,VER,1,0 days 00:01:36.296000,2.0,1.0,NaT,NaT,0 days 00:00:30.916000,...,,False,True,62,0 days 01:02:14.121000,18.3,49.0,1017.0,False,23.8
1,2,0 days 01:04:50.559000,VER,1,0 days 00:01:36.753000,3.0,1.0,NaT,NaT,0 days 00:00:30.999000,...,,False,True,63,0 days 01:03:14.127000,18.3,49.0,1017.1,False,23.8
2,3,0 days 01:06:27.206000,VER,1,0 days 00:01:36.647000,4.0,1.0,NaT,NaT,0 days 00:00:30.931000,...,,False,True,65,0 days 01:05:14.127000,18.3,49.0,1017.0,False,23.7
3,4,0 days 01:08:04.379000,VER,1,0 days 00:01:37.173000,5.0,1.0,NaT,NaT,0 days 00:00:31.255000,...,,False,True,67,0 days 01:07:14.127000,18.3,50.0,1017.1,False,23.5
4,5,0 days 01:09:41.471000,VER,1,0 days 00:01:37.092000,6.0,1.0,NaT,NaT,0 days 00:00:31.041000,...,,False,True,68,0 days 01:08:14.122000,18.3,50.0,1017.1,False,23.5
5,6,0 days 01:11:18.509000,VER,1,0 days 00:01:37.038000,7.0,1.0,NaT,NaT,0 days 00:00:31.015000,...,,False,True,70,0 days 01:10:14.127000,18.3,50.0,1017.1,False,23.5
6,7,0 days 01:12:55.533000,VER,1,0 days 00:01:37.024000,8.0,1.0,NaT,NaT,0 days 00:00:31.041000,...,,False,True,72,0 days 01:12:14.127000,18.2,50.0,1017.2,False,23.7
7,8,0 days 01:14:32.762000,VER,1,0 days 00:01:37.229000,9.0,1.0,NaT,NaT,0 days 00:00:31.100000,...,,False,True,73,0 days 01:13:14.142000,18.2,50.0,1017.2,False,23.5
8,11,0 days 01:19:23.852000,VER,1,0 days 00:01:37.045000,12.0,1.0,NaT,NaT,0 days 00:00:30.966000,...,,False,True,78,0 days 01:18:14.133000,18.2,50.0,1017.2,False,23.5
9,12,0 days 01:21:00.882000,VER,1,0 days 00:01:37.030000,13.0,1.0,NaT,NaT,0 days 00:00:30.964000,...,,False,True,80,0 days 01:20:14.140000,18.3,50.0,1017.2,False,23.5


In [ ]:
for drv in s.laps['Driver'].unique():
    laps = s.laps.pick_wo_box().pick_compounds()